# Custom Clients

In [1]:
import os
from pprint import pprint as pp

os.environ["LOG_LEVEL"] = "debug"

LANGGATE_URL = "http://localhost:4000/api/v1"

# If running the langgate server behind Envoy proxy in docker:
# LANGGATE_URL = "http://localhost:10000/api/v1"

# If running and accessing langgate from within a kubernetes cluster:
# LANGGATE_URL = "http://langgate.ns.svc.cluster.local:10000/api/v1"

## Custom Local Registry Client

In [2]:
# If you wan to set a custom config file:
# os.environ["LANGGATE_CONFIG"] = "some_custom_path/langgate_config.yaml"
# os.environ["LANGGATE_CONFIG"] = "some_custom_path/langgate_config.yaml"

### Example 1: Using the SDK's default local registry client

In [10]:
from langgate.sdk import LangGateLocal

# `LangGateLocal.registry` is a singleton
client = LangGateLocal()
models = await client.list_llms()  # returns the default LLMInfo schema list
pp(models[0])

LLMInfo(id='openai/gpt-5-chat', name='ChatGPT-5', provider_id='openai', description='ChatGPT-5 from OpenAI is designed for advanced, natural, multimodal, and context-aware conversations.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per_token=Decimal('0.00001'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('1.25E-7'), input_cached_cost_per_token=None), capabilities=ModelCapabilities(supports_tools=False, supports_parallel_tool_calls=None, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_reasoning=True, supports_response_schema=True, supports_system_messages=None, supports_max_tokens=True, supports_files=True, supports_seed=True), context_window=ContextWindow(max_input_tokens=400000, max_output_tokens=128000), provider=ModelProvider(id='openai', name='OpenAI', description=None), updated_dt=datetime.datetime(2025, 8, 18, 12, 55, 52,

### Example 2: Using the standalone default local registry client
The registry can be installed without the SDK if you do not need to use any other SDK features such as transforming paramaters.
It can be installed separately with:
```bash
uv add langgate[registry]
```
or with pip:
```bash
pip install langgate[registry]
```

In [3]:
from langgate.registry.local import LocalRegistryClient

# The concrete `LocalRegistryClient` class is a singleton
client = LocalRegistryClient()
models = await client.list_llms()  # returns the default LLMInfo schema list
pp(models[0])

LLMInfo(id='openai/gpt-5-chat', name='ChatGPT-5', provider_id='openai', description='ChatGPT-5 from OpenAI is designed for advanced, natural, multimodal, and context-aware conversations.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per_token=Decimal('0.00001'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('1.25E-7'), input_cached_cost_per_token=None), capabilities=ModelCapabilities(supports_tools=False, supports_parallel_tool_calls=None, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_reasoning=True, supports_response_schema=True, supports_system_messages=None, supports_max_tokens=True, supports_files=True, supports_seed=True), context_window=ContextWindow(max_input_tokens=400000, max_output_tokens=128000), provider=ModelProvider(id='openai', name='OpenAI', description=None), updated_dt=datetime.datetime(2025, 8, 18, 12, 55, 52,

### Example 3: Subclassing BaseLocalRegistryClient with custom schema

In [4]:
from langgate.registry.local import BaseLocalRegistryClient
from langgate.core.models import LLMInfo, ImageModelInfo


class CustomLLMInfo(LLMInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomImageModelInfo(ImageModelInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomLocalRegistryClient(
    BaseLocalRegistryClient[CustomLLMInfo, CustomImageModelInfo]
):
    """Custom Local registry client with both CustomLLMInfo and CustomImageModelInfo schemas.

    Note: If you only want to extend one schema, pass the default class for the other type parameter.
    For example: BaseLocalRegistryClient[CustomLLMInfo, ImageModelInfo]
    """

    # This is not a singleton unless you implement it as such.


custom_client = CustomLocalRegistryClient()
custom_llms = (
    await custom_client.list_llms()
)  # Typed and validated as list[CustomLLMInfo]
pp(custom_llms[0])

custom_image_models = (
    await custom_client.list_image_models()
)  # Typed and validated as list[CustomImageModelInfo]
pp(custom_image_models[0])

2025-08-18 13:56:10 [debug    ] reusing_registry_singleton     initialized=True
2025-08-18 13:56:10 [debug    ] initialized_base_local_registry_client
2025-08-18 13:56:10 [debug    ] refreshing_model_caches       
2025-08-18 13:56:10 [debug    ] refreshed_model_caches         image_count=4 llm_count=5
CustomLLMInfo(id='openai/gpt-5-chat', name='ChatGPT-5', provider_id='openai', description='ChatGPT-5 from OpenAI is designed for advanced, natural, multimodal, and context-aware conversations.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per_token=Decimal('0.00001'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('1.25E-7'), input_cached_cost_per_token=None), capabilities=ModelCapabilities(supports_tools=False, supports_parallel_tool_calls=None, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_reasoning=True, supports_response_schema=

### Example 4: Using the base class directly with type parameter

In [5]:
# This is not a singleton.
local_client_with_custom_schema = BaseLocalRegistryClient[
    CustomLLMInfo, CustomImageModelInfo
](llm_info_cls=CustomLLMInfo, image_info_cls=CustomImageModelInfo)
direct_llms = (
    await local_client_with_custom_schema.list_llms()
)  # Typed and validated as list[CustomLLMInfo]

pp(direct_llms[0])

direct_image_models = (
    await local_client_with_custom_schema.list_image_models()
)  # Typed and validated as list[CustomImageModelInfo]
pp(direct_image_models[0])

2025-08-18 13:56:17 [debug    ] reusing_registry_singleton     initialized=True
2025-08-18 13:56:17 [debug    ] initialized_base_local_registry_client
2025-08-18 13:56:17 [debug    ] refreshing_model_caches       
2025-08-18 13:56:17 [debug    ] refreshed_model_caches         image_count=4 llm_count=5
CustomLLMInfo(id='openai/gpt-5-chat', name='ChatGPT-5', provider_id='openai', description='ChatGPT-5 from OpenAI is designed for advanced, natural, multimodal, and context-aware conversations.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per_token=Decimal('0.00001'), input_cost_per_token_batches=None, output_cost_per_token_batches=None, cache_read_input_token_cost=Decimal('1.25E-7'), input_cached_cost_per_token=None), capabilities=ModelCapabilities(supports_tools=False, supports_parallel_tool_calls=None, supports_vision=True, supports_audio_input=None, supports_audio_output=None, supports_prompt_caching=True, supports_reasoning=True, supports_response_schema=

## Custom HTTP Registry Client

In [6]:
from pydantic import SecretStr

# Optional: Set API key (if your LangGate service requires authentication)
api_key = SecretStr("your_api_key")

### Example 1: Using the default http client

In [7]:
from langgate.client.http import HTTPRegistryClient

# The concrete `HTTPRegistryClient` class is a singleton
client = HTTPRegistryClient(base_url=LANGGATE_URL, api_key=api_key)
llms = await client.list_llms()  # returns the default LLMInfo schema list
pp(llms[0])

2025-08-18 13:56:20 [debug    ] creating_http_registry_client_singleton
2025-08-18 13:56:20 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 image_info_cls=<class 'langgate.core.models.ImageModelInfo'> llm_info_cls=<class 'langgate.core.models.LLMInfo'>
2025-08-18 13:56:20 [debug    ] initialized_http_registry_client_singleton
2025-08-18 13:56:20 [debug    ] refreshing_model_caches       
2025-08-18 13:56:20 [debug    ] refreshed_model_caches         image_count=33 llm_count=65
LLMInfo(id='openai/gpt-5', name='GPT-5', provider_id='openai', description='GPT-5 is OpenAI\'s most advanced model, offering major improvements in reasoning, code quality, and user experience. It is optimized for complex tasks that require step-by-step reasoning, instruction following, and accuracy in high-stakes use cases. It supports user-specified intent like "think hard about this." Improvements include reductions in hallucination, sycoph

### Example 2: Subclassing BaseHTTPRegistryClient with custom schema

In [8]:
from langgate.client.http import BaseHTTPRegistryClient
from langgate.core.models import LLMInfo, ImageModelInfo


class CustomLLMInfo(LLMInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomImageModelInfo(ImageModelInfo):
    extra_field: str = ""
    custom_metadata: dict = {}


class CustomHTTPClient(BaseHTTPRegistryClient[CustomLLMInfo, CustomImageModelInfo]):
    """Custom HTTP client with both CustomLLMInfo and CustomImageModelInfo schemas.

    Note: If you only want to extend one schema, pass the default class for the other type parameter.
    For example: BaseHTTPRegistryClient[CustomLLMInfo, ImageModelInfo]
    """

    # This is not a singleton unless you implement it as such.


custom_client = CustomHTTPClient(LANGGATE_URL, api_key=api_key)
custom_models = (
    await custom_client.list_llms()
)  # Typed and validated as list[CustomLLMInfo]
pp(custom_models[0])

custom_image_models = (
    await custom_client.list_image_models()
)  # Typed and validated as list[CustomImageModelInfo]
pp(custom_image_models[0])

2025-08-18 13:56:25 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 image_info_cls=<class '__main__.CustomImageModelInfo'> llm_info_cls=<class '__main__.CustomLLMInfo'>
2025-08-18 13:56:25 [debug    ] refreshing_model_caches       
2025-08-18 13:56:25 [debug    ] refreshed_model_caches         image_count=33 llm_count=65
CustomLLMInfo(id='openai/gpt-5', name='GPT-5', provider_id='openai', description='GPT-5 is OpenAI\'s most advanced model, offering major improvements in reasoning, code quality, and user experience. It is optimized for complex tasks that require step-by-step reasoning, instruction following, and accuracy in high-stakes use cases. It supports user-specified intent like "think hard about this." Improvements include reductions in hallucination, sycophancy, and better performance in coding, writing, and health-related tasks.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per

### Example 3: Using the base class directly with type parameter

In [9]:
# This is not a singleton.
client_with_custom_schema = BaseHTTPRegistryClient[CustomLLMInfo, CustomImageModelInfo](
    base_url=LANGGATE_URL,
    api_key=api_key,
    llm_info_cls=CustomLLMInfo,
    image_info_cls=CustomImageModelInfo,
)
direct_llms = (
    await client_with_custom_schema.list_llms()
)  # Typed and validated as list[CustomLLMInfo]

pp(direct_llms[0])

direct_image_models = (
    await client_with_custom_schema.list_image_models()
)  # Typed and validated as list[CustomImageModelInfo]
pp(direct_image_models[0])

2025-08-18 13:56:27 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 image_info_cls=<class '__main__.CustomImageModelInfo'> llm_info_cls=<class '__main__.CustomLLMInfo'>
2025-08-18 13:56:27 [debug    ] refreshing_model_caches       
2025-08-18 13:56:27 [debug    ] refreshed_model_caches         image_count=33 llm_count=65
CustomLLMInfo(id='openai/gpt-5', name='GPT-5', provider_id='openai', description='GPT-5 is OpenAI\'s most advanced model, offering major improvements in reasoning, code quality, and user experience. It is optimized for complex tasks that require step-by-step reasoning, instruction following, and accuracy in high-stakes use cases. It supports user-specified intent like "think hard about this." Improvements include reductions in hallucination, sycophancy, and better performance in coding, writing, and health-related tasks.', costs=TokenCosts(input_cost_per_token=Decimal('0.00000125'), output_cost_per